In [1]:
from comet_ml import Experiment
import argparse
from tqdm import tqdm
import os
import torch
import torch.nn as nn
import torch.optim as optim

from head.metrics import CosFace
from loss.focal import FocalLoss
from util.utils import separate_resnet_bn_paras, warm_up_lr, load_checkpoint, \
    schedule_lr, AverageMeter, accuracy
from util.fairness_utils import evaluate
from util.data_utils_balanced import *
import numpy as np
import pandas as pd
import random
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(222)
torch.cuda.manual_seed_all(222)
np.random.seed(222)
random.seed(222)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

default_test_root = '/cmlscratch/sdooley1/data/CelebA/Img/img_align_celeba_splits/test/'
default_train_root = '/cmlscratch/sdooley1/data/CelebA/Img/img_align_celeba_splits/train/'


In [10]:
parser = argparse.ArgumentParser()

parser.add_argument('--data_test_root', default=default_test_root)
parser.add_argument('--data_train_root', default=default_train_root)
parser.add_argument('--demographics', default= '/cmlscratch/sdooley1/data/CelebA/CelebA_demographics.txt')
parser.add_argument('--backbone_name', default='resnet50')
parser.add_argument('--pretrained', default=False)
parser.add_argument('--project_name', default="from-scratch_no-resampling_adam")

parser.add_argument('--checkpoints_root', default='/cmlscratch/sdooley1/Checkpoints/timm_explore_few_epochs/')
parser.add_argument('--head_name', default='CosFace')
parser.add_argument('--train_loss', default='Focal', type=str)

parser.add_argument('--groups_to_modify', default= ['male', 'female'], type=str, nargs='+')
parser.add_argument('--p_identities', default=[1.0, 1.0], type=float, nargs='+')
parser.add_argument('--p_images', default=[1.0, 1.0], type=float, nargs='+')
parser.add_argument('--min_num_images', default=3, type=int)

parser.add_argument('--batch_size', default=250, type=int)
parser.add_argument('--input_size', default=112, type=int)
parser.add_argument('--weight_decay', default=5e-4, type=float)
parser.add_argument('--momentum', default=0.9, type=float)
parser.add_argument('--mean', default=[0.5, 0.5, 0.5], type=int)
parser.add_argument('--std', default=[0.5, 0.5, 0.5], type=int)
parser.add_argument('--stages', default=[35, 65, 95], type=int)
parser.add_argument('--num_workers', default=4, type=int)

parser.add_argument('--lr', default=0.001, type=float)
parser.add_argument('--num_epoch', default=3, type=int)
parser.add_argument('--gpu_id', default=[0], type=int, nargs='+', help='gpu id')
parser.add_argument('--name', default='CelebA', type=str)
parser.add_argument('--dataset', default='CelebA', type=str)
parser.add_argument('--file_name', default='timm_from-scratch.csv', type=str)
parser.add_argument('--seed', default=222, type=int)

args = parser.parse_args(['--backbone_name','resnet101'])

p_images = {args.groups_to_modify[i]:args.p_images[i] for i in range(len(args.groups_to_modify))}
p_identities = {args.groups_to_modify[i]:args.p_identities[i] for i in range(len(args.groups_to_modify))}
args.p_images = p_images
args.p_identities = p_identities

print("P identities: {}".format(args.p_identities))
print("P images: {}".format(args.p_images))




P identities: {'male': 1.0, 'female': 1.0}
P images: {'male': 1.0, 'female': 1.0}


In [11]:
dataloaders, num_class, demographic_to_labels_train, demographic_to_labels_test = prepare_data(args)



''' Model '''
backbone = timm.create_model(args.backbone_name, 
                             num_classes=0,
                             pretrained=args.pretrained).to(device)
config = timm.data.resolve_data_config({}, model=backbone)
model_input_size = config['input_size']

# get model's embedding size
meta = pd.read_csv('/cmlscratch/sdooley1/timm_model_metadata.csv')
embedding_size = int(meta[meta['model_name'] == args.backbone_name].feature_dim)



head = CosFace(in_features=embedding_size, out_features=num_class, device_id=args.gpu_id)
train_criterion = FocalLoss(elementwise=True)

####################################################################################################################
# ======= optimizer =======#

backbone_paras_only_bn, backbone_paras_wo_bn = separate_resnet_bn_paras(backbone)
_, head_paras_wo_bn = separate_resnet_bn_paras(head)
optimizer = optim.Adam([{'params': backbone_paras_wo_bn + head_paras_wo_bn, 'weight_decay': args.weight_decay},
                           {'params': backbone_paras_only_bn}], lr=args.lr)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=100)


backbone, head = backbone.to(device), head.to(device)
backbone, head, optimizer, epoch, batch, checkpoints_model_root = load_checkpoint(args, backbone, head, optimizer, dataloaders['train'], p_identities, p_images)
backbone = nn.DataParallel(backbone)
backbone = backbone.to(device)

PREPARING TRAIN DATASET
Overall # of images for male available is 67562
# images selected for male is 67562
Overall # of images for female available is 76524
# images selected for female is 67562
Number of idx for male is 3529
Number of idx for female is 3529
PREPARING TEST DATASET
Overall # of images for male available is 7644
# images selected for male is 7636
Overall # of images for female available is 8851
# images selected for female is 7636
Number of idx for male is 406
Number of idx for female is 406
Len of train dataloader is 540
Len of test dataloader is 62
Checkpoint_Head_CosFace_Backbone_resnet101_Dataset_CelebA_p_idx{'male': 1.0, 'female': 1.0}_p_img{'male': 1.0, 'female': 1.0}_Epoch_
Found checkpoints for this model: []
No Checkpoints Found at '/cmlscratch/sdooley1/Checkpoints/timm_explore_few_epochs/resnet101_CosFace'. Please Have a Check or Continue to Train from Scratch


In [14]:
_,_, acc_k_test, intra_test, inter_test, _,_,_,_,_ = evaluate(dataloaders['test'], train_criterion, backbone, head, embedding_size,
                           k_accuracy = True, multilabel_accuracy = False,
                           demographic_to_labels = demographic_to_labels_test, test = True)

results = {}
results['Model'] = args.backbone_name
results['seed'] = 222
for k in acc_k_test.keys():
#                 experiment.log_metric("Loss Train " + k, loss_train[k], step=epoch)
#                 experiment.log_metric("Acc Train " + k, acc_train[k], step=epoch)
    results['Acc '+k] = (round(acc_k_test[k].item()*100, 3))
    results['Intra '+k] = (round(intra_test[k], 3))
    results['Inter '+k] = (round(intra_test[k], 3))
    results['epoch'] = epoch

print(results)
#save_output_from_dict('results_nooversampling', results, args.file_name)


100%|██████████| 62/62 [00:18<00:00,  3.32it/s]


{'Model': 'resnet101', 'seed': 222, 'Acc male': 7.059, 'Intra male': 7.937, 'Inter male': 7.937, 'epoch': 0, 'Acc female': 6.168, 'Intra female': 7.406, 'Inter female': 7.406}


In [15]:
loss_train, acc_train, _, _, _, _,_,_,_,_ = evaluate(dataloaders['train'], train_criterion, backbone,head, embedding_size,
                                    k_accuracy = False, multilabel_accuracy = True,
                                    demographic_to_labels = demographic_to_labels_train, test = False)


100%|██████████| 540/540 [01:55<00:00,  4.68it/s]


In [20]:
# training eval
dataloader, criterion, backbone, head, emb_size = dataloaders['train'], train_criterion, backbone,head, embedding_size
k_accuracy = False
multilabel_accuracy = True
demographic_to_labels = demographic_to_labels_train
test = False

loss = {k:torch.tensor(0.0) for k in demographic_to_labels.keys()}
acc = {k:torch.tensor(0.0) for k in demographic_to_labels.keys()}
count = {k:torch.tensor(0.0) for k in demographic_to_labels.keys()}
acc_k = {k:torch.tensor(0.0) for k in demographic_to_labels.keys()}
intra = {k:torch.tensor(0.0) for k in demographic_to_labels.keys()}
inter = {k:torch.tensor(0.0) for k in demographic_to_labels.keys()}
angles_intra, angles_inter, correct = 0, 0, 0

backbone.eval()
if multilabel_accuracy:
    head.eval()

# figure out embedding size
if emb_size is None:
    inputs, _, _ = next(iter(dataloader))
    x = torch.randn(inputs.shape).to(device)
    emb_size = backbone(x).shape[1]


feature_matrix = torch.empty(0, emb_size)
labels_all = []
demographic_all = []

for inputs, labels, sens_attr in tqdm(iter(dataloader)):
    inputs = inputs.to(device)
    labels = labels.to(device).long()
    sens_attr = np.array(sens_attr)
    with torch.no_grad():

        if multilabel_accuracy:
            features = backbone(inputs)
            outputs = head(features, labels)
            loss_value = criterion(outputs, labels)

            # add sum of losses for female and male images
            for k in loss.keys():
                loss[k] += loss_value[sens_attr == k].sum().cpu()

            # multiclass accuracy
            _, predicted = outputs.max(1)
            for k in acc.keys():
                acc[k] +=  predicted[sens_attr == k].eq(labels[sens_attr == k]).sum().cpu().item()

            for k in count.keys():
                count[k] += sum(sens_attr == k)

        if k_accuracy:
            #need to build feature matrix
            inputs_flipped = torch.flip(inputs, [3])
            embed = backbone(inputs) + backbone(inputs_flipped)
            features_batch = l2_norm(embed)
            feature_matrix = torch.cat((feature_matrix, features_batch.detach().cpu()), dim = 0)

            labels_all = labels_all + labels.cpu().tolist()
            demographic_all = demographic_all + sens_attr.tolist()



100%|██████████| 540/540 [01:48<00:00,  4.99it/s]


In [26]:
# _, acc_test, _, _, _, _,_,_,_,_ = evaluate(dataloaders['test'], train_criterion, backbone,head, embedding_size,
#                                     k_accuracy = False, multilabel_accuracy = True,
#                                     demographic_to_labels = demographic_to_labels_train, test = False)
_,acc_test, acc_k_test, intra_test, inter_test, _,_,_,_,_ = evaluate(dataloaders['test'], train_criterion, backbone, head, embedding_size, k_accuracy = True, multilabel_accuracy = True, demographic_to_labels = demographic_to_labels_test, test = True)

results = {}
results['Model'] = args.backbone_name
results['seed'] = args.seed
results['epoch'] = -1
for k in acc_k_test.keys():
#     experiment.log_metric("Acc multi Test " + k, acc_test[k], step=-1)
#     experiment.log_metric("Acc k Test " + k, acc_k_test[k], step=-1)
#     experiment.log_metric("Intra Test " + k, intra_test[k], step=-1)
#     experiment.log_metric("Inter Test " + k, inter_test[k], step=-1)

    results['Acc multi '+k] = (round(acc_test[k].item()*100, 3))
    results['Acc k '+k] = (round(acc_k_test[k].item()*100, 3))
    results['Intra '+k] = (round(intra_test[k], 3))
    results['Inter '+k] = (round(inter_test[k], 3))

print(results)


100%|██████████| 62/62 [00:27<00:00,  2.28it/s]


{'Model': 'resnet101', 'seed': 222, 'epoch': -1, 'Acc multi male': 0.0, 'Acc k male': 7.059, 'Intra male': 7.937, 'Inter male': 1.832, 'Acc multi female': 0.0, 'Acc k female': 6.168, 'Intra female': 7.406, 'Inter female': 1.696}


In [21]:
{'Model': 'resnet101', 'seed': 222, 'epoch': -1, 
 'Acc multi male': 0.0, 'Acc k male': 7.059, 'Intra male': 7.937, 'Inter male': 1.832, 
 'Acc multi female': 0.0, 'Acc k female': 6.168, 'Intra female': 7.406, 'Inter female': 1.696}


(tensor([[[[-0.6000, -0.5922, -0.5765,  ..., -0.0980, -0.1137, -0.1373],
           [-0.6000, -0.5922, -0.5765,  ..., -0.1529, -0.1529, -0.1686],
           [-0.6078, -0.6000, -0.5922,  ..., -0.2157, -0.2157, -0.2314],
           ...,
           [-0.2471, -0.3020, -0.3490,  ...,  0.1765,  0.1843,  0.1922],
           [-0.3176, -0.3490, -0.3569,  ...,  0.1922,  0.2000,  0.2078],
           [-0.3412, -0.3569, -0.3647,  ...,  0.2157,  0.2314,  0.2392]],
 
          [[-0.7490, -0.7412, -0.7333,  ..., -0.4902, -0.4980, -0.5137],
           [-0.7490, -0.7412, -0.7333,  ..., -0.5529, -0.5529, -0.5529],
           [-0.7569, -0.7490, -0.7490,  ..., -0.6078, -0.6078, -0.6078],
           ...,
           [-0.5608, -0.6235, -0.6706,  ..., -0.4118, -0.3961, -0.3804],
           [-0.6157, -0.6471, -0.6549,  ..., -0.3961, -0.3804, -0.3647],
           [-0.6235, -0.6392, -0.6471,  ..., -0.3725, -0.3490, -0.3333]],
 
          [[-0.8039, -0.7961, -0.8039,  ..., -0.6549, -0.6627, -0.6784],
           [-

In [19]:
acc_train

{'male': tensor(0.), 'female': tensor(0.)}